In [1]:
import torch
import os
import sys
from re import I
os.environ["CUDA_VISIBLE_DEVICES"]="2"
%env CUDA_VISIBLE_DEVICES=2
torch.cuda.set_device(2)
device = 'cuda:2'
sys.path.append('../../')
sys.path.append('../../video_diffusion_pytorch/')
from vq_gan_3d.model.vqgan import VQGAN
from vq_gan_3d.dataset import MRNetDataset, BRATSDataset
import matplotlib.pyplot as plt
import SimpleITK as sitk
from video_diffusion_pytorch import Unet3D, GaussianDiffusion, Trainer
import pytorch_ssim



/data/home/firas/anaconda3/envs/vq_gan_3d/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/home/firas/anaconda3/envs/vq_gan_3d/lib/python3.8/site-packages/torch/cuda/__init__.py:146: UserWarning: 
NVIDIA RTX A6000 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA RTX A6000 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


env: CUDA_VISIBLE_DEVICES=2


In [2]:
val_dataset = BRATSDataset(root_dir='/data/BraTS/BraTS 2020', train=False, imgtype='flair')

trainset = BRATSDataset(root_dir='/data/BraTS/BraTS 2020', train=True, imgtype='flair')
train_loader = torch.utils.data.DataLoader(trainset, batch_size = 2, shuffle=True, num_workers=10)

In [3]:

vqgan_ckpt = '/data/home/firas/Desktop/work/other_groups/video_diffusion_pytorch/vqgan_checkpoints/brats/epoch=728-step=153000-train/recon_loss=0.06.ckpt'

model = Unet3D(
    dim=32,
    dim_mults=(1, 2, 4, 8),
    channels=8,
).cuda()

diffusion = GaussianDiffusion(
    model,
    vqgan_ckpt=vqgan_ckpt,
    image_size=32,
    num_frames=32,
    channels=8,
    timesteps=300,           # number of steps
    # number of sampling timesteps (using ddim for faster inference [see citation for ddim paper])
    # sampling_timesteps=250,
    loss_type='l1',            # L1 or L2
    # objective=args.objective
).cuda()

trainer = Trainer(
    diffusion,
    args=None,
    dataset=val_dataset,
    train_batch_size=1,
    save_and_sample_every=10,
    train_num_steps=700000,         # total training steps
    gradient_accumulate_every=2,    # gradient accumulation steps
    ema_decay=0.995,                # exponential moving average decay
    amp=False,                       # turn on mixed precision
    num_sample_rows=1
    # logger=logger
)

trainer.load('/data/home/firas/Desktop/work/other_groups/video_diffusion_pytorch/results/model-82.pt')

/data/home/firas/anaconda3/envs/vq_gan_3d/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/data/home/firas/anaconda3/envs/vq_gan_3d/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loaded pretrained LPIPS loss from /data/home/firas/Desktop/work/other_groups/vq_gan_3d/model/cache/vgg.pth
found 118 videos as gif files at None


In [4]:
sum_ssim = 0
for k in range(20):
    for i,dat in enumerate(train_loader):
        dat = dat['data']
        if len(dat)!=2:
            print("Length: ", len(dat))
            break
        img1 = dat[0]
        img2 = dat[1]

        msssim = pytorch_ssim.msssim_3d(img1,img2)
        sum_ssim = sum_ssim+msssim
    print(sum_ssim/((k+1)*(i+1)))

tensor(0.7385)
tensor(0.7440)
tensor(0.7443)


In [9]:
sum_ssim = 0
for i in range(1000):
    trainer.ema_model.eval()
    with torch.no_grad():
        sample = trainer.ema_model.sample(batch_size=2)
    img1 = sample[0].cpu()
    img2 = sample[1].cpu()

    msssim = pytorch_ssim.msssim_3d(img1,img2)
    sum_ssim = sum_ssim+msssim
    print(sum_ssim/(i+1.0))
print(sum_ssim/1000)

sampling loop time step:  99%|█████████▉| 297/300 [00:18<00:00, 15.97it/s]


KeyboardInterrupt: 